In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import math

import warnings
warnings.filterwarnings("ignore")

def stack2dataframe(df,index_nm,column_nm,feature_nm):
    """privot table"""
    return pd.pivot_table(
        df,
        index = index_nm,
        columns = column_nm,
        values = feature_nm).sort_index()

def get_weights(m,T,tn):
    w = m**(1/T)
    """生成weight序列"""
    w_list = np.array([w**(tn-1-i) for i in range(tn)])
    return w_list


def get_Njyr(end_date,N,df_date ):
    return sorted((df_date[df_date['tradingdate']<=end_date]['tradingdate'].values))[-N:]

In [48]:
df_hq = pd.read_csv('result_20150328_20220328.csv')

In [49]:
df_hq.shape

(6822127, 14)

In [50]:
df_hq.head()

Unnamed: 0        date       code     close  preclose        volume  \
0           0  2015-03-30  sh.000001  3786.568  3691.096  5.647024e+10   
1           1  2015-03-31  sh.000001  3747.899  3786.568  5.616760e+10   
2           2  2015-04-01  sh.000001  3810.294  3747.899  4.474583e+10   
3           3  2015-04-02  sh.000001  3825.784  3810.294  4.792997e+10   
4           4  2015-04-03  sh.000001  3863.929  3825.784  4.730333e+10   

         amount  adjustflag      turn  tradestatus    pctChg  isST       pct  \
0  6.921254e+11           3  0.022085            1  2.586553   NaN  1.025865   
1  7.212949e+11           3  0.021965            1 -1.021220   NaN  0.989788   
2  5.924183e+11           3  0.017498            1  1.664800   NaN  1.016648   
3  6.320289e+11           3  0.018742            1  0.406530   NaN  1.004065   
4  6.356513e+11           3  0.018496            1  0.997051   NaN  1.009971   

    pct_abs  
0  0.025865  
1  0.010212  
2  0.016648  
3  0.004065  
4  0.009971

In [51]:
len(df_hq['code'].unique())

5179

In [52]:
df_hq.groupby('date')['code'].count()

date
2015-03-30    3070
2015-03-31    3070
2015-04-01    3070
2015-04-02    3070
2015-04-03    3070
              ... 
2022-03-22    5173
2022-03-23    5175
2022-03-24    5176
2022-03-25    5178
2022-03-28    5179
Name: code, Length: 1705, dtype: int64

In [53]:
df_hq.drop(['Unnamed: 0','adjustflag'],axis = 1,inplace = True)
df_hq.rename(columns = {'pctChg':'changepct','turn':'turnoverrate','date':'tradingday','code':'secucode'},inplace = True)

In [54]:
df_hq.columns

Index(['tradingday', 'secucode', 'close', 'preclose', 'volume', 'amount',
       'turnoverrate', 'tradestatus', 'changepct', 'isST', 'pct', 'pct_abs'],
      dtype='object')

In [29]:
# df_hq['isST'].unique()
# df_hq[df_hq['isST']==0]['tradestatus'].unique()
# df_hq[df_hq['isST']==1]['tradestatus'].unique()
# df_hq[df_hq['isST'].isnull()]['tradestatus'].unique()
# df_hq[df_hq['amount']>0]['tradestatus'].unique()
# df_hq[df_hq['amount']>0]['isST'].unique()
# df_hq['adjustflag'].value_counts()

In [55]:
df_hq = df_hq[df_hq['amount']>0].copy()
market_dict = {
    'sh':['600','601','603','605','688'],
    'sz':['000','001','002','003','300','301']}

df_hq['secucategory'] = df_hq['secucode'].apply(lambda x: 1 if x.split('.')[1][:3] in market_dict[x.split('.')[0]] else 0) 

In [56]:
df_hq['tradingday'] = df_hq['tradingday'].apply(lambda x:x.replace('-',''))
df_hq['secucode'] = df_hq['secucode'].apply(lambda x:x.split('.')[1])
df_300 = df_hq[df_hq['secucode']=='000300'].copy()
df_hq = df_hq[df_hq['secucategory']==1].copy()

In [57]:
df_hq[df_hq['secucode']=='000300']

tradingday secucode      close   preclose        volume        amount  \
238355   20150330   000300  4088.1780  3971.6970  4.282970e+10  5.462111e+11   
238356   20150331   000300  4051.2040  4088.1780  4.360656e+10  5.981217e+11   
238357   20150401   000300  4123.8950  4051.2040  3.295129e+10  4.665969e+11   
238358   20150402   000300  4124.7760  4123.8950  3.390705e+10  4.766178e+11   
238359   20150403   000300  4170.5380  4124.7760  3.213038e+10  4.669656e+11   
...           ...      ...        ...        ...           ...           ...   
240055   20220322   000300  4255.2950  4258.7459  1.263825e+10  2.262135e+11   
240056   20220323   000300  4276.5174  4255.2950  1.170891e+10  2.263908e+11   
240057   20220324   000300  4251.3061  4276.5174  1.089183e+10  2.195944e+11   
240058   20220325   000300  4174.5742  4251.3061  1.088080e+10  2.170132e+11   
240059   20220328   000300  4148.4663  4174.5742  1.184490e+10  2.134690e+11   

        turnoverrate  tradestatus  changepct  isST       pct   pct_abs  \
238355      0.019437            1   2.932775   NaN  1.029328  0.029328   
238356      0.019789            1  -0.904410   NaN  0.990956  0.009044   
238357      0.014954            1   1.794304   NaN  1.017943  0.017943   
238358      0.015388            1   0.021360   NaN  1.000214  0.000214   
238359      0.014581            1   1.109447   NaN  1.011094  0.011094   
...              ...          ...        ...   ...       ...       ...   
240055      0.451850            1  -0.081031   0.0  0.999190  0.000810   
240056      0.418660            1   0.498729   0.0  1.004987  0.004987   
240057      0.389205            1  -0.589529   0.0  0.994105  0.005895   
240058      0.388811            1  -1.804902   0.0  0.981951  0.018049   
240059      0.423222            1  -0.625403   0.0  0.993746  0.006254   

        secucategory  
238355             0  
238356             0  
238357             0  
238358             0  
238359             0  
...              ...  
240055             0  
240056             0  
240057             0  
240058             0  
240059             0  

[1705 rows x 13 columns]

In [58]:
df_hq[(df_hq['tradingday']=='20210316')&(df_hq['secucode']=='000300')].copy()

tradingday secucode      close   preclose        volume        amount  \
239808   20210316   000300  5079.3624  5035.5441  1.613901e+10  2.717550e+11   

        turnoverrate  tradestatus  changepct  isST       pct   pct_abs  \
239808      0.581782            1    0.87018   0.0  1.008702  0.008702   

        secucategory  
239808             0

In [11]:
df_date = pd.read_csv('tradingday.csv')
df_date['tradingdate'] = df_date['tradingdate'].astype(str)

In [124]:
# get_Njyr('20210101',1040+11+273)

In [136]:
date_list = sorted([col for col in df_date['tradingdate'].unique() if col >='20210101' and col <='20211231'])

In [135]:
for dt in date_list:
    print(dt,'Begin=====')
    #dt = '20220328'
    df_res = df_hq[df_hq['tradingday']==dt][['tradingday','secucode']].copy()

    N = 252
    dt_list = get_Njyr(dt,N)
    turnoverrate = stack2dataframe(df_hq[(df_hq['tradingday'].isin(dt_list))].copy(),'tradingday','secucode','turnoverrate')
    w_list = get_weights(0.5,63,252)
    weights = turnoverrate.copy()
    weights.loc[:,:] = np.repeat(
        w_list.reshape(252,1),len(turnoverrate.columns),axis = 1)
    annualized_traded_value_ratio = (weights * turnoverrate ).rolling(252,min_periods = 1).sum()/np.sum(weights)
    annualized_traded_value_ratio = dict(annualized_traded_value_ratio.loc[dt,:])
    df_res['annualized_traded_value_ratio'] = df_res['secucode'].map(annualized_traded_value_ratio)

    """
    average_turnoverrate_1m: 计算过去21个交易日的换手率之和,
    average_turnoverrate_1q: 计算过去63个交易日的换手率之和,
    average_turnoverrate_1yr: 计算过去252个交易日的换手率之和,
    """
    N = 21
    dt_list = get_Njyr(dt,N)
    turnoverrate = stack2dataframe(df_hq[(df_hq['tradingday'].isin(dt_list))].copy(),'tradingday','secucode','turnoverrate')
    average_turnoverrate_1m = np.log(np.nansum(turnoverrate/100,axis = 0))
    average_turnoverrate_1m = {k:v for k,v in zip(turnoverrate.columns,average_turnoverrate_1m)}
    df_res['average_turnoverrate_1m'] = df_res['secucode'].map(average_turnoverrate_1m)

    N = 63
    dt_list = get_Njyr(dt,N)
    turnoverrate = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','turnoverrate')
    stom_list = []
    for i in range(0,N,21):
        stom = np.log(np.nansum(turnoverrate[turnoverrate.index[i]:turnoverrate.index[i+20]]/100,axis = 0))
        stom_list.append(stom)
    average_turnoverrate_1q = np.log(np.nanmean(np.exp(np.stack(stom_list,axis = 0)),axis = 0))
    average_turnoverrate_1q = {k:v for k,v in zip(turnoverrate.columns,average_turnoverrate_1q)}
    df_res['average_turnoverrate_1q'] = df_res['secucode'].map(average_turnoverrate_1q)

    N = 252
    dt_list = get_Njyr(dt,N)
    turnoverrate = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','turnoverrate')
    stom_list = []
    for i in range(0,N,21):
        stom = np.log(np.nansum(turnoverrate[turnoverrate.index[i]:turnoverrate.index[i+20]]/100,axis = 0))
        stom_list.append(stom)
    average_turnoverrate_1yr = np.log(np.nanmean(np.exp(np.stack(stom_list,axis = 0)),axis = 0))
    average_turnoverrate_1yr = {k:v for k,v in zip(turnoverrate.columns,average_turnoverrate_1yr)}

    df_res['average_turnoverrate_1yr'] = df_res['secucode'].map(average_turnoverrate_1yr)

    """
    daily_std_dec: 日收益率在过去252个交易日的波动率，半衰期42个交易日,
    """
    N = 252
    dt_list = get_Njyr(dt,N)
    changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','changepct')/100
    changepct = changepct.fillna({k:v for k,v in zip(changepct.columns,np.nanmean(changepct,axis = 0))})
    """
    dastd = sqrt(sum(wi*(ri-avg(ri))^2))
    """
    w = (1/2)**(1/42)
    tn = 252
    """生成weight序列"""
    w_list = np.array([w**(tn-1-i) for i in range(tn)])
    weights = np.repeat(w_list.reshape(N,1),len(changepct.columns),axis = 1)
    daily_std_dec = (np.nansum((changepct.values - np.nanmean(changepct,axis = 0))**2  * weights,axis = 0)/\
    np.nansum(weights,axis = 0))**0.5

    daily_std_dec = {k:v for k,v in zip(changepct.columns,daily_std_dec)}

    df_res['daily_std_dec'] = df_res['secucode'].map(daily_std_dec)

    """hist_sigma: 在计算BETA所进行的时间序列回归中，取回归残差收益率的波动率,"""

    index_pct_list = np.array(df_300[df_300['tradingday'].isin(dt_list)].sort_values('tradingday')['changepct'])/100
    index_pct = np.repeat(index_pct_list.reshape(N,1),len(changepct.columns),axis = 1)
    x = changepct.values
    y = index_pct_list.copy()
    Y = sm.add_constant(y)
    model = sm.WLS(x,Y,w_list)
    result = model.fit()
    # alpha_ ,beta_  = result.params
    resid_ = result.resid
    hist_sigma = np.nanstd(resid_,axis = 0)
    hist_sigma = {k:v for k,v in zip(changepct.columns,hist_sigma)}

    df_res['hist_sigma'] = df_res['secucode'].map(hist_sigma)

    """
    beta_1y: 
    股票收益率 对沪深300收益率，进行时间序列回归，取回归系数，回归时间窗口为252个交易日，半衰期63个交易日,
    historical_alpha: 
    股票收益率 对沪深300收益率 进行时间序列回归，取截距项，回归时间窗口为252个交易日，半衰期63个交易日,
    """
    N = 252
    dt_list = get_Njyr(dt,N)
    changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','changepct')/100
    changepct = changepct.fillna({k:v for k,v in zip(changepct.columns,np.nanmean(changepct,axis = 0))})
    w = 0.5**(1/63)
    tn = len(changepct.index)
    """生成weight序列"""
    w_list = np.array([w**(tn-1-i) for i in range(tn)])

    x = changepct.values
    y = index_pct_list.copy()
    Y = sm.add_constant(y)
    model = sm.WLS(x,Y,w_list)
    result = model.fit()
    alpha_ ,beta_  = result.params

    historical_alpha = {k:v for k,v in zip(changepct.columns,alpha_)}
    beta_1y = {k:v for k,v in zip(changepct.columns,beta_)}
    df_res['historical_alpha'] = df_res['secucode'].map(historical_alpha)
    df_res['beta_1y'] = df_res['secucode'].map(beta_1y)

    """
    decl_dr_wgt_mean_12_month: 最近n个月以每日换手率为权重，计算每日收益率的算术平均值，n=12，换手率随时间衰减,
    decl_dr_wgt_mean_6_month: 最近n个月以每日换手率为权重，计算每日收益率的算术平均值，n=6，换手率随时间衰减,
    """

    N = 6*20
    dt_list = get_Njyr(dt,N)

    turnoverrate = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','turnoverrate')
    changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','changepct')/100
    w = (1/math.e)**(1/(120*4))
    tn = N

    """生成weight序列"""
    w_list = np.array([w**(tn-1-i) for i in range(tn)])

    weigths = turnoverrate.values * np.repeat(w_list.reshape(N,1),len(changepct.columns),axis =1)
    decl_dr_wgt_mean_6_month = np.nansum(weigths * changepct.values ,axis = 0)/np.nansum(weigths,axis = 0)
    decl_dr_wgt_mean_6_month = {k:v for k,v in zip(changepct.columns,decl_dr_wgt_mean_6_month)}

    df_res['decl_dr_wgt_mean_6_month'] = df_res['secucode'].map(decl_dr_wgt_mean_6_month)

    N = 12*20
    dt_list = get_Njyr(dt,N)

    turnoverrate = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','turnoverrate')
    changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','changepct')/100

    w = (1/math.e)**(1/(N*4))
    tn = N
    """生成weight序列"""
    w_list = np.array([w**(tn-1-i) for i in range(tn)])
    weigths = turnoverrate.values * np.repeat(w_list.reshape(N,1),len(changepct.columns),axis =1)
    decl_dr_wgt_mean_12_month = np.nansum(weigths * changepct.values ,axis = 0)/np.nansum(weigths,axis = 0)
    decl_dr_wgt_mean_12_month = {k:v for k,v in zip(changepct.columns,decl_dr_wgt_mean_12_month)}

    df_res['decl_dr_wgt_mean_12_month'] = df_res['secucode'].map(decl_dr_wgt_mean_12_month)

    """
    dr_abs_mean_val_mean_12m: 过去12月收益率绝对值均值/成交额均值,
    dr_abs_mean_val_mean_1m: 过去1月收益率绝对值均值/成交额均值,
    dr_abs_mean_val_mean_3m: 过去3月收益率绝对值均值/成交额均值,
    dr_abs_mean_val_mean_6m: 过去6月收益率绝对值均值/成交额均值,
    """
    N = 12*20
    dt_list = get_Njyr(dt,N)

    turnovervalue = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','amount')/10000
    changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','changepct')/100


    dr_abs_mean_val_mean_12m = np.nanmean(abs(changepct),axis = 0)*100000/np.nanmean(turnovervalue,axis = 0)
    dr_abs_mean_val_mean_12m = {k:v for k,v in zip(changepct.columns,dr_abs_mean_val_mean_12m)}
    df_res['dr_abs_mean_val_mean_12m'] = df_res['secucode'].map(dr_abs_mean_val_mean_12m)

    N = 1*20
    dt_list = get_Njyr(dt,N)

    turnovervalue = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','amount')/10000
    changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','changepct')/100

    dr_abs_mean_val_mean_1m = np.nanmean(abs(changepct),axis = 0)*100000/np.nanmean(turnovervalue,axis = 0)
    dr_abs_mean_val_mean_1m = {k:v for k,v in zip(changepct.columns,dr_abs_mean_val_mean_1m)}

    df_res['dr_abs_mean_val_mean_1m'] = df_res['secucode'].map(dr_abs_mean_val_mean_1m)

    N = 3*20
    dt_list = get_Njyr(dt,N)

    turnovervalue = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','amount')/10000
    changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','changepct')/100

    dr_abs_mean_val_mean_3m = np.nanmean(abs(changepct),axis = 0)*100000/np.nanmean(turnovervalue,axis = 0)
    dr_abs_mean_val_mean_3m = {k:v for k,v in zip(changepct.columns,dr_abs_mean_val_mean_3m)}

    df_res['dr_abs_mean_val_mean_3m'] = df_res['secucode'].map(dr_abs_mean_val_mean_3m)

    N = 6*20
    dt_list = get_Njyr(dt,N)

    turnovervalue = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','amount')/10000
    changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','changepct')/100
    dr_abs_mean_val_mean_6m = np.nanmean(abs(changepct),axis = 0)*100000/np.nanmean(turnovervalue,axis = 0)
    dr_abs_mean_val_mean_6m = {k:v for k,v in zip(changepct.columns,dr_abs_mean_val_mean_6m)}

    df_res['dr_abs_mean_val_mean_6m'] = df_res['secucode'].map(dr_abs_mean_val_mean_6m)

    """
    turn_val_std_mean_12m: 过去12月成交额标准差/均值,
    turn_val_std_mean_1m: 过去1月成交额标准差/均值,
    turn_val_std_mean_3m: 过去3月成交额标准差/均值,
    turn_val_std_mean_6m: 过去6月成交额标准差/均值,
    """
    N = 12*20
    dt_list = get_Njyr(dt,N)

    turnovervalue = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','amount')/10000
    changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','changepct')/100


    turn_val_std_mean_12m = np.nanstd(turnovervalue,ddof = 1,axis = 0) / np.nanmean(turnovervalue,axis = 0)
    turn_val_std_mean_12m = {k:v for k,v in zip(changepct.columns,turn_val_std_mean_12m)}

    df_res['turn_val_std_mean_12m'] = df_res['secucode'].map(turn_val_std_mean_12m)

    N = 1*20
    dt_list = get_Njyr(dt,N)

    turnovervalue = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','amount')/10000
    changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','changepct')/100



    turn_val_std_mean_1m = np.nanstd(turnovervalue,ddof = 1,axis = 0) / np.nanmean(turnovervalue,axis = 0)
    turn_val_std_mean_1m = {k:v for k,v in zip(changepct.columns,turn_val_std_mean_1m)}

    df_res['turn_val_std_mean_1m'] = df_res['secucode'].map(turn_val_std_mean_1m)

    N = 3*20
    dt_list = get_Njyr(dt,N)

    turnovervalue = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','amount')/10000
    changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','changepct')/100


    turn_val_std_mean_3m = np.nanstd(turnovervalue,ddof = 1,axis = 0) / np.nanmean(turnovervalue,axis = 0)
    turn_val_std_mean_3m = {k:v for k,v in zip(changepct.columns,turn_val_std_mean_3m)}

    df_res['turn_val_std_mean_3m'] = df_res['secucode'].map(turn_val_std_mean_3m)

    N = 6*20
    dt_list = get_Njyr(dt,N)

    turnovervalue = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','amount')/10000
    changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','changepct')/100

    turn_val_std_mean_6m = np.nanstd(turnovervalue,ddof = 1,axis = 0) / np.nanmean(turnovervalue,axis = 0)
    turn_val_std_mean_6m = {k:v for k,v in zip(changepct.columns,turn_val_std_mean_6m)}

    df_res['turn_val_std_mean_6m'] = df_res['secucode'].map(turn_val_std_mean_6m)


    """
    std_12m: 过去12月日收益率标准差,
    std_3m: 过去3月日收益率标准差,
    std_6m: 过去6月日收益率标准差,
    """
    N = 12*20
    dt_list = get_Njyr(dt,N)
    changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','changepct')/100
    std_12m = np.nanstd(changepct,ddof = 1,axis = 0) 
    std_12m = {k:v for k,v in zip(changepct.columns,std_12m*100)}

    df_res['std_12m'] = df_res['secucode'].map(std_12m)

    N = 3*20
    dt_list = get_Njyr(dt,N)
    changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','changepct')/100
    std_3m = np.nanstd(changepct,ddof = 1,axis = 0) 
    std_3m = {k:v for k,v in zip(changepct.columns,std_3m*100)}

    df_res['std_3m'] = df_res['secucode'].map(std_3m)

    N = 6*20
    dt_list = get_Njyr(dt,N)
    changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','changepct')/100
    std_6m = np.nanstd(changepct,ddof = 1,axis = 0) 
    std_6m = {k:v for k,v in zip(changepct.columns,std_6m*100)}

    df_res['std_6m'] = df_res['secucode'].map(std_6m)

    """
    holding_return_1m: 最近21个交易日的累计持仓收益率,
    """
    N = 21
    dt_list = get_Njyr(dt,N)
    changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','changepct')/100
    changepct_ = np.nancumprod(1+changepct,axis = 0)

    holding_return_1m = np.log(changepct_)[-1,:] -1
    holding_return_1m = {k:v for k,v in zip(changepct.columns,holding_return_1m)}

    df_res['holding_return_1m'] = df_res['secucode'].map(holding_return_1m)

    """
    relative_strength: 
    1、计算非滞后的相对强度 对股票的对数收益率进行半衰指数加权求和，时间窗口252个交易日，半衰期126个交易日；
    2、以11个交易日为时间窗口，滞后11个交易日，取非滞后相对强度的等权平均值,
    """

    N = 252
    dt_list = get_Njyr(dt,N+11+11)


    changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','changepct')/100

    changepct = np.log(1+ changepct)# 对数收益率
    w = (1/2)**(1/126)
    tn = N
    """生成weight序列"""
    w_list = np.array([w**(N-1-i) for i in range(N)])
    weights = np.repeat(w_list.reshape(N,1),len(changepct.columns),axis = 1)
    rs_list = []
    for i in range(11):
        rs = changepct.loc[changepct.index[-(252+11+i)]: changepct.index[-(12+i)]].values * weights
        rs_list.append(np.nansum(rs * 10000,axis = 0)/np.nansum(weights,axis = 0))

    relative_strength = np.nanmean(np.stack(rs_list,axis = 0),axis = 0)
    relative_strength = {k:v for k,v in zip(changepct.columns,relative_strength)}

    df_res['relative_strength'] = df_res['secucode'].map(relative_strength)

    """
    long_term_relative_strength: 
    1、计算非滞后的长期相对强度，对股票对数收益率进行加权求和，时间窗口1040个交易日，半衰期260个交易日；
    2、滞后273个交易日，在11个交易日的时间窗口内取非滞后值等权平均值，最后取相反数,
    """
    N = 1040
    dt_list = get_Njyr(dt,N+11+273)
    changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','changepct')/100
    changepct = np.log(1+ changepct)# 对数收益率

    w = (1/2)**(1/260)
    """生成weight序列"""
    w_list = np.array([w**(N-1-i) for i in range(N)])

    weights = np.repeat(w_list.reshape(N,1),len(changepct.columns),axis = 1)

    rs_list = []
    for i in range(11):
        rs = changepct.loc[changepct.index[-(N+273+i)]: changepct.index[-(273+1+i)]].values * weights
        rs_list.append(-np.nansum(rs * 10000,axis = 0)/np.nansum(weights,axis = 0))
    long_term_relative_strength = np.nanmean(np.stack(rs_list,axis = 0),axis = 0)
    long_term_relative_strength = {k:v for k,v in zip(changepct.columns,long_term_relative_strength)}

    df_res['long_term_relative_strength'] = df_res['secucode'].map(long_term_relative_strength)

    """
    long_term_historical_alpha: 
    1、股票收益率 对沪深300收益率，进行时间序列回归，取回归系数，回归时间窗口为1040个交易日，半衰期260个交易日 ；
    2、取回归截距项，之后273个交易日，在11个交易日的时间窗口内取非滞后值等全平均值，最后取相反数,
    """

    N = 1040
    dt_list = get_Njyr(dt,N+11+273)
    changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','changepct')/100
    changepct = np.log(1+ changepct)# 对数收益率

    index_pct_list = np.array(df_300[df_300['tradingday'].isin(dt_list)].sort_values('tradingday')['changepct'])/100
    w = 0.5**(1/260)
    """生成weight序列"""
    w_list = np.array([w**(N-1-i) for i in range(N)])
    # weights = np.repeat(w_list.reshape(N,1),len(changepct.columns),axis = 1)

    alpha_list = []
    for i in range(11):
        x = changepct.loc[changepct.index[-(N+273+i)]: changepct.index[-(273+1+i)]].values 
        y = index_pct_list[-(N+273+i):-(273+i)]
        Y = sm.add_constant(y)
        model = sm.WLS(x,Y,w_list)
        result = model.fit()
        alpha_ ,beta_  = result.params
        alpha_list.append(alpha_)

    long_term_historical_alpha = -np.nanmean(np.stack(alpha_list,axis = 0),axis = 0)*10**6
    long_term_historical_alpha = {k:v for k,v in zip(changepct.columns,long_term_historical_alpha)}
    df_res['long_term_historical_alpha'] = df_res['secucode'].map(long_term_historical_alpha)

    """
    short_term_revarsal: 最近一个月的加权累积对数日收益率，时间窗口21个交易日，半衰期5个交易日，T={t-1，.....，t-n},
    """
    N = 21
    dt_list = get_Njyr(dt,N)
    changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','changepct')/100

    w = 0.5**(1/5)
    w_list = np.array([w**(N-1-i) for i in range(N)])
    weights = np.repeat(w_list.reshape(N,1),len(changepct.columns),axis = 1)
    short_term_revarsal = np.nansum(changepct * weights,axis = 0)/ np.nansum(weights,axis=0) * 10000
    short_term_revarsal = {k:v for k,v in zip(changepct.columns,short_term_revarsal)}

    df_res['short_term_revarsal'] = df_res['secucode'].map(short_term_revarsal)

    """
    dr_abs_val_mean_log_20d: 过去20 日（每日涨跌幅绝对值/成交额）的均值取对数,
    """
    N = 20
    dt_list = get_Njyr(dt,N)
    turnovervalue = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','amount')/10000
    changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','changepct')/100

    dr_abs_val_mean_log_20d = np.log(np.mean(abs(changepct)/turnovervalue*100000,axis = 0))
    dr_abs_val_mean_log_20d = {k:v for k,v in zip(changepct.columns,dr_abs_val_mean_log_20d)}

    df_res['dr_abs_val_mean_log_20d'] = df_res['secucode'].map(dr_abs_val_mean_log_20d)

    # df_res[df_res['secucode'].isin(['000001','002001','003000','300001','301000','600000','601001','605001','688001'])].sort_values('secucode')

    df_res.to_csv('df_res_{}.csv'.format(dt),index= None)

In [3]:
import os 

file_list = [col for col in os.listdir('F:/南大实习') if 'df_res' in col ]

df_list = []
for file_ in file_list:
    df_list.append(pd.read_csv('F:/南大实习/'+file_))

df_result = pd.concat(df_list)
df_result.shape
# df_result.to_csv('F:/南大实习/df_result_{}_{}.csv'.format(date_list[0],date_list[-1]),index = None)



(2624364, 29)

In [5]:
df_result.head()

tradingday  secucode  annualized_traded_value_ratio  \
0    20210104    600000                       0.177095   
1    20210104    600004                       0.938421   
2    20210104    600006                       3.601800   
3    20210104    600007                       0.251752   
4    20210104    600008                       0.572542   

   average_turnoverrate_1m  average_turnoverrate_1q  average_turnoverrate_1yr  \
0                -3.485384                -3.196405                 -3.418906   
1                -2.162102                -1.723062                 -1.614041   
2                 0.540058                 0.081271                 -0.822145   
3                -3.513784                -3.240397                 -2.867258   
4                -2.338368                -2.248811                 -2.144780   

   daily_std_dec  hist_sigma  historical_alpha   beta_1y  ...  \
0       0.011217    0.009225         -0.001427  0.657311  ...   
1       0.020497    0.020530         -0.001513  0.785064  ...   
2       0.040336    0.028065          0.005639  0.738977  ...   
3       0.014859    0.014606         -0.002015  0.734948  ...   
4       0.011124    0.012000         -0.001372  0.552089  ...   

   turn_val_std_mean_6m   std_12m    std_3m    std_6m  holding_return_1m  \
0              0.455910  1.391080  1.010629  1.081610          -1.055209   
1              0.642062  2.485429  1.978049  1.955248          -1.073446   
2              1.344840  3.101549  4.549812  3.558259          -0.625504   
3              0.690019  1.872656  1.211226  1.583762          -1.048792   
4              0.553942  1.566930  0.856861  1.309984          -1.064322   

   relative_strength  long_term_relative_strength  long_term_historical_alpha  \
0          -3.802548                    -2.902941                         NaN   
1          -3.020786                    -7.889663                 -457.130291   
2          23.500513                     4.564931                         NaN   
3          -6.973244                    -3.326502                 -131.854588   
4          -1.026197                     6.456878                         NaN   

   short_term_revarsal  dr_abs_val_mean_log_20d  
0             3.203864                -4.250377  
1            19.627084                -2.523571  
2           105.295670                -2.832770  
3            -9.081989                -0.656950  
4            -2.477899                -2.679367  

[5 rows x 29 columns]

In [6]:
df_result.isnull().sum()

tradingday                             0
secucode                               0
annualized_traded_value_ratio          0
average_turnoverrate_1m                0
average_turnoverrate_1q                0
average_turnoverrate_1yr               0
daily_std_dec                          0
hist_sigma                             0
historical_alpha                       0
beta_1y                                0
decl_dr_wgt_mean_6_month               0
decl_dr_wgt_mean_12_month              0
dr_abs_mean_val_mean_12m               0
dr_abs_mean_val_mean_1m                0
dr_abs_mean_val_mean_3m                0
dr_abs_mean_val_mean_6m                0
turn_val_std_mean_12m               1116
turn_val_std_mean_1m                1118
turn_val_std_mean_3m                1116
turn_val_std_mean_6m                1116
std_12m                             1116
std_3m                              1116
std_6m                              1116
holding_return_1m                      0
relative_strengt

In [9]:
df_result['tradingday'] = df_result['tradingday'].astype(str)
df_result['secucode'] = df_result['secucode'].apply(lambda x:str(x).zfill(6))

In [11]:
df_result= df_result[(df_result['tradingday']>='20210101') & (df_result['tradingday']<='20211231')].copy()

In [13]:
isnull_dict = dict(df_result.isnull().sum()/df_result.shape[0])

In [16]:
# isnull_dict
feature_columns = [c for c in df_result.columns if c not in ['tradingday','secucode'] and isnull_dict[c]<=0.1]

In [18]:
df_result[feature_columns].describe()

annualized_traded_value_ratio  average_turnoverrate_1m  \
count                   2.115172e+06             2.115172e+06   
mean                    2.807680e+00            -9.938914e-01   
std                     2.494774e+00             1.001347e+00   
min                     5.065431e-03            -9.581404e+00   
25%                     1.112615e+00            -1.685785e+00   
50%                     2.035346e+00            -1.022935e+00   
75%                     3.654276e+00            -3.264146e-01   
max                     2.277395e+01             2.407558e+00   

       average_turnoverrate_1q  average_turnoverrate_1yr  daily_std_dec  \
count             2.115172e+06              2.115172e+06   2.115172e+06   
mean             -9.513682e-01             -8.531285e-01   3.636514e-02   
std               9.472467e-01              8.293187e-01   4.730679e-02   
min              -6.538493e+00             -7.924788e+00   0.000000e+00   
25%              -1.602236e+00             -1.393096e+00   2.221987e-02   
50%              -9.641063e-01             -8.102908e-01   2.859829e-02   
75%              -3.139590e-01             -2.629877e-01   3.648620e-02   
max               2.059690e+00              1.497840e+00   2.225815e+00   

         hist_sigma  historical_alpha       beta_1y  decl_dr_wgt_mean_6_month  \
count  2.115172e+06      2.115172e+06  2.115172e+06              2.115172e+06   
mean   3.593975e-02      4.448690e-03  4.999374e-01              1.435566e-02   
std    4.185542e-02      6.794655e-02  4.589463e-01              9.688462e-02   
min    0.000000e+00     -2.727270e-01 -8.378875e+00             -2.727270e-01   
25%    2.269424e-02     -8.742129e-04  2.624720e-01              7.088169e-04   
50%    2.807146e-02      2.599039e-04  4.461294e-01              3.699502e-03   
75%    3.485217e-02      2.046760e-03  6.991702e-01              7.450540e-03   
max    1.218766e+00      1.942581e+01  1.864817e+01              1.942581e+01   

       decl_dr_wgt_mean_12_month  ...  turn_val_std_mean_3m  \
count               2.115172e+06  ...          2.114204e+06   
mean                1.766574e-02  ...          6.627663e-01   
std                 9.936514e-02  ...          2.543292e-01   
min                -2.727270e-01  ...          4.543187e-05   
25%                 1.982989e-03  ...          4.869661e-01   
50%                 4.394439e-03  ...          6.059200e-01   
75%                 7.477021e-03  ...          7.779290e-01   
max                 1.942581e+01  ...          4.504313e+00   

       turn_val_std_mean_6m       std_12m        std_3m        std_6m  \
count          2.114204e+06  2.114204e+06  2.114204e+06  2.114204e+06   
mean           7.919984e-01  4.765044e+00  3.812535e+00  4.273579e+00   
std            2.845659e-01  1.132987e+01  1.017588e+01  1.095194e+01   
min            4.543187e-05  1.010456e-01  1.010456e-01  1.010456e-01   
25%            5.916282e-01  2.376612e+00  2.063514e+00  2.183652e+00   
50%            7.374475e-01  2.951442e+00  2.732762e+00  2.818533e+00   
75%            9.310169e-01  3.680730e+00  3.570275e+00  3.589915e+00   
max            4.523395e+00  1.391837e+03  1.391837e+03  1.391837e+03   

       holding_return_1m  relative_strength  long_term_relative_strength  \
count       2.115172e+06       2.115172e+06                 2.115172e+06   
mean       -9.840863e-01       4.450074e+00                -1.477664e+00   
std         1.658995e-01       2.143938e+01                 1.017594e+01   
min        -2.417156e+00      -1.108006e+02                -8.432775e+01   
25%        -1.069688e+00      -8.421476e+00                -5.651765e+00   
50%        -9.983573e-01       1.087370e-01                 0.000000e+00   
75%        -9.216958e-01       1.326712e+01                 4.413930e+00   
max         2.016799e+00       1.912293e+02                 5.379123e+01   

       short_term_revarsal  dr_abs_val_mean_log_20d  
count         2.115172e+06             2.115172e+0

In [36]:
"""归一化"""
def minmaxstandardlize(x):
    """
    df_macd['macd1'] = df_macd.groupby(['date','stock_id'])['macd']\
    .transform(minmaxstandardlize)
    """
    x = list(x)
    x_max = np.nanmax(x)
    x_min = np.nanmin(x)
    return pd.Series([(c-x_min)/(x_max - x_min) if (x_max - x_min)>0 else 0 for c in x])



def roll_minmaxstandardlize(list1):
    """截面归一化"""
    x_max = np.nanmax(list1)
    x_min = np.nanmin(list1)
    return pd.Series((list1[-1]-x_min)/(x_max - x_min) if (x_max - x_min)>0 else 0)


def Zscorestandardlize(x):
    """
    df_macd['macd1'] = df_macd.groupby(['date','stock_id'])['macd']\
    .transform(Zscorestandardlize)
    """
    x = list(x)
    x_mean = np.nanmean(x)
    x_std = np.nanstd(x)
    return pd.Series([(c-x_mean)/(x_std) if x_std!=0 else 0 for c in x])





In [21]:
np.nanpercentile(np.array([1,2,3,4,5,7,7,8,9,10]),97.5)

9.775

In [22]:
feature_columns_ = [c+'_' for c in feature_columns]

In [91]:
test = df_result[(df_result['secucode'].isin(['000001','000002']))&(df_result['tradingday']<='20210110')].copy()

In [92]:
test

tradingday secucode  annualized_traded_value_ratio  \
1779    20210104   000001                       0.568522   
1779    20210104   000001                       0.568522   
5892    20210105   000001                       0.573014   
1780    20210105   000001                       0.573014   
10004   20210106   000001                       0.578113   
1779    20210106   000001                       0.578113   
14120   20210107   000001                       0.581142   
1780    20210107   000001                       0.581142   
18240   20210108   000001                       0.581817   
1781    20210108   000001                       0.581817   
1780    20210104   000002                       0.783866   
1780    20210104   000002                       0.783866   
5893    20210105   000002                       0.788587   
1781    20210105   000002                       0.788587   
10005   20210106   000002                       0.792079   
1780    20210106   000002                       0.792079   
14121   20210107   000002                       0.797584   
1781    20210107   000002                       0.797584   
18241   20210108   000002                       0.800701   
1782    20210108   000002                       0.800701   

       average_turnoverrate_1m  average_turnoverrate_1q  \
1779                 -2.361104                -2.192625   
1779                 -2.361104                -2.192625   
5892                 -2.311535                -2.182840   
1780                 -2.311535                -2.182840   
10004                -2.257832                -2.171960   
1779                 -2.257832                -2.171960   
14120                -2.216114                -2.161708   
1780                 -2.216114                -2.161708   
18240                -2.191921                -2.167632   
1781                 -2.191921                -2.167632   
1780                 -2.020780                -1.929845   
1780                 -2.020780                -1.929845   
5893                 -1.993961                -1.912279   
1781                 -1.993961                -1.912279   
10005                -1.962797                -1.899380   
1780                 -1.962797                -1.899380   
14121                -1.911834                -1.887022   
1781                 -1.911834                -1.887022   
18241                -1.884935                -1.884708   
1782                 -1.884935                -1.884708   

       average_turnoverrate_1yr  daily_std_dec  hist_sigma  historical_alpha  \
1779                  -2.107622       0.020243    0.016144          0.000357   
1779                  -2.107622       0.020243    0.016144          0.000357   
5892                  -2.103471       0.020315    0.016588         -0.000016   
1780                  -2.103471       0.020315    0.016588         -0.000016   
10004                 -2.099191       0.022361    0.016942          0.000722   
1779                  -2.099191       0.022361    0.016942          0.000722   
14120                 -2.095257       0.022274    0.016926          0.000732   
1780                  -2.095257       0.022274    0.016926          0.000732   
18240                 -2.092536       0.022094    0.016920          0.000732   
1781                  -2.092536       0.022094    0.016920          0.000732   
1780                  -1.739295       0.015718    0.016070         -0.001103   
1780                  -1.739295       0.015718    0.016070         -0.001103   
5893                  -1.737980       0.015597    0.016042         -0.001184   
1781                  -1.737980       0.015597    0.016042         -0.001184   
10005                 -1.735982       0.015944    0.016030         -0.000907   
1780                  -1.735982       0.015944    0.016030         -0.000907   
14121                 -1.733688       0.015809    0.016019         -0.000977   
1781                  -1.733688       0.015809    0.016019       

In [31]:
x=test[['annualized_traded_value_ratio','average_turnoverrate_1m']].values.copy()
h = 97.5
l= 2.5
np.nanpercentile(x,h,0)
# .transform(filterbypercentile,97.5,2.5)

array([ 0.5818166 , -2.19192067])

In [34]:
# filterbypercentile(test[['annualized_traded_value_ratio','average_turnoverrate_1m']],97.5,2.5)

In [166]:

def filterbypercentile(X,h,l):
    x = X.values.copy()
    h_ = np.nanpercentile(x,h,0)
    l_ = np.nanpercentile(x,l,0)
    h_ = np.repeat(h_.reshape(1,-1),x.shape[0],axis = 0)
    l_ = np.repeat(l_.reshape(1,-1),x.shape[0],axis = 0)
    #print(type(x))
    x[x>h_] = h_[x>h_]
    x[x<l_] = l_[x<l_]
    return [x]

t = test.groupby(['secucode'])[feature_columns].apply(lambda x:filterbypercentile(x,97.5,2.5))
test[feature_columns_] = np.array([col[0] for col in t.values]).reshape(-1,26)

In [163]:
# test[feature_columns].values - test[feature_columns_].values

In [134]:
df_result = df_result.sort_values(['secucode','tradingday'])
tmp_ =df_result.groupby(['secucode'])[feature_columns].apply(lambda x:filterbypercentile(x,97.5,2.5))

In [174]:
t = [np.array(x).flatten().tolist() for x in tmp_.values]

In [180]:
t1 = sum(t,[])

In [182]:
df_result[feature_columns_] = np.array(t1).reshape(-1,26)

MemoryError: 

In [164]:
# df_result[feature_columns_] = 
np.array([np.array(x).reshape(-1,26) for x in tmp_.values]).flatten().tolist()[1].shape

(486, 26)

In [136]:
np.array([col[0] for col in tmp_.values]).shape
# .reshape(-1,len(feature_columns))

(4599,)

In [133]:

df_result[feature_columns_] = np.array([col[0] for col in tmp_.values]).reshape(-1,len(feature_columns))

ValueError: cannot reshape array of size 4599 into shape (26)

In [ ]:
df_result[feature_columns_] - df_result[feature_columns]

In [77]:
test[feature_columns_] = filterbypercentile(test[feature_columns],97.5,2.5)

In [89]:
# np.nanpercentile(test[feature_columns[0:2]].values,20,0)

In [81]:
test[feature_columns_].values - test[feature_columns].values

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 

In [70]:
test[feature_columns[0]].values

array([0.56852223, 0.57301405, 0.57811267, 0.58114179, 0.5818166 ,
       0.56852223, 0.57301405, 0.57811267, 0.58114179, 0.5818166 ])

In [66]:
t[0][0].shape

(10, 26)

In [57]:
x_list

NameError: name 'x_list' is not defined

In [23]:
"""异常值截断"""
df_result[feature_columns_] = df_result.groupby(['secucode'])[feature_columns].transform(filterbypercentile,97.5,2.5)

IndexError: boolean index did not match indexed array along dimension 1; dimension is 1 but corresponding boolean dimension is 486